In [1]:
import os

# Define directory structure
dirs = [
    "data",
    "models",
    "optim",
    "loss",
    "utils",
    "experiments"
]

# Define empty files to create
files = [
    "train.py",
    "config.py",
    "requirements.txt",
    "README.md",
    "data/dataloader.py",
    "models/layers.py",
    "models/neuralnet.py",
    "optim/optimizers.py",
    "loss/loss_functions.py",
    "utils/metrics.py",
    "utils/wandb_utils.py",
    "experiments/sweep_config.yaml"
]

# Create directories
for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)

# Create empty files
for file_path in files:
    with open(file_path, 'w') as f:
        f.write("# " + os.path.basename(file_path))

print("✅ Project structure created successfully.")


✅ Project structure created successfully.


conda command
conda create -n dl_cpu_env python=3.10 pytorch torchvision torchaudio cpuonly numpy pandas -c pytorch -c conda-forge


In [2]:
import torch
w=torch.randn((30,3))
x=torch.randn((3,))
b=torch.randn((30,))


In [6]:
x.shape,w.shape,b.shape

(torch.Size([3]), torch.Size([30, 3]), torch.Size([30]))

In [4]:
torch.equal(w@x+b,x@w.T+b)

True